In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [ ]:
path_to_file = "scripts.txt"

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 13631487 characters


In [ ]:
print(text[:250])

    TWELVE MONKEYS
     
            An original screenplay by

    David Peoples
           &
    Janet Peoples

     Inspired by
     
   LA JETEE, a Chris Marker Film





 Production Draft
 June 27, 1994








 FADE IN


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

95 unique characters


In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[66, 67, 68, 69, 70, 71, 72], [89, 90, 91]]>

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(13631487,), dtype=int64, numpy=array([ 4,  4,  4, ...,  4, 69, 74])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

 
 
 
 
T
W
E
L
V
E


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b' ' b' ' b' ' b' ' b'T' b'W' b'E' b'L' b'V' b'E' b' ' b'M' b'O' b'N'
 b'K' b'E' b'Y' b'S' b'\r' b'\n' b' ' b' ' b' ' b' ' b' ' b'\r' b'\n' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b'A' b'n' b' '
 b'o' b'r' b'i' b'g' b'i' b'n' b'a' b'l' b' ' b's' b'c' b'r' b'e' b'e'
 b'n' b'p' b'l' b'a' b'y' b' ' b'b' b'y' b'\r' b'\n' b'\r' b'\n' b' ' b' '
 b' ' b' ' b'D' b'a' b'v' b'i' b'd' b' ' b'P' b'e' b'o' b'p' b'l' b'e'
 b's' b'\r' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b'&' b'\r' b'\n'], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'    TWELVE MONKEYS\r\n     \r\n            An original screenplay by\r\n\r\n    David Peoples\r\n           &\r\n'
b'    Janet Peoples\r\n\r\n     Inspired by\r\n     \r\n   LA JETEE, a Chris Marker Film\r\n\r\n\r\n\r\n\r\n\r\n Production'
b' Draft\r\n June 27, 1994\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n FADE IN:\r\n\r\n INT.  CONCOURSE/AIRPORT TERMINAL - BAY\r\n\r\n CLOS'
b'E ON A FACE.  A nine year old boy, YOUNG COLE, his eyes wide\r\n with wonder. watching something intent'
b'ly.  We HEAR the sounds of\r\n the P.A. SYSTEM droning Flight Information mingled with the\r\n sounds of '


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'    TWELVE MONKEYS\r\n     \r\n            An original screenplay by\r\n\r\n    David Peoples\r\n           &\r'
Target: b'   TWELVE MONKEYS\r\n     \r\n            An original screenplay by\r\n\r\n    David Peoples\r\n           &\r\n'


In [ ]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 97) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  24832     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

array([ 5, 54, 66, 44, 57, 94, 21, 68, 28, 57, 96, 65, 10, 18,  4,  2, 48,
       40, 36, 16,  9, 12,  4,  3, 90, 39, 24,  9, 82, 57, 48, 29, 80, 32,
        6, 28, 24, 73, 21, 27, 36, 70, 80, 22, 36,  6, 71, 62, 69, 75, 43,
       89, 15, 94, 70,  2,  5, 96, 49, 40, 82,  6, 73, 62, 88,  3, 82, 67,
       31,  0, 14, 82, 92, 92, 59, 19, 83, 19, 79, 75, 67, 62, 20, 66, 60,
       49, 76, 91, 82,  5, 36, 15, 56, 86,  9, 75, 22, 26, 12, 66])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ORD/FREEWAY\r\n\r\n A ROLODEX CARD with an address on "Outerbridge Road" for "Jeffrey\r\n Mason c/o Dr. Ma'

Next Char Predictions:
 b'!SaIV}1c8V\xc2\xb0`&. \nMEA,%( \ryD4%qVM9o<"84h17Aeo2A"f[djHx+}e\n!\xc2\xb0NEq"h[w\rqb;*q{{X/r/njb[0aYNkzq!A+Uu%j26(a'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.574714


In [ ]:
tf.exp(mean_loss).numpy()

97.000305

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './deneme2'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 50

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
2108/2108 [==============================] - 113s 53ms/step - loss: 1.7256
Epoch 2/50
2108/2108 [==============================] - 118s 56ms/step - loss: 1.0203
Epoch 3/50
2108/2108 [==============================] - 120s 57ms/step - loss: 0.9422
Epoch 4/50
2108/2108 [==============================] - 120s 57ms/step - loss: 0.9048
Epoch 5/50
2108/2108 [==============================] - 120s 57ms/step - loss: 0.8828
Epoch 6/50
2108/2108 [==============================] - 120s 57ms/step - loss: 0.8679
Epoch 7/50
2108/2108 [==============================] - 120s 57ms/step - loss: 0.8581
Epoch 8/50
2108/2108 [==============================] - 120s 57ms/step - loss: 0.8518
Epoch 9/50
2108/2108 [==============================] - 120s 57ms/step - loss: 0.8473
Epoch 10/50
2108/2108 [==============================] - 120s 57ms/step - loss: 0.8454
Epoch 11/50
2108/2108 [==============================] - 119s 57ms/step - loss: 0.8439
Epoch 12/50
2108/2108 [=============================

KeyboardInterrupt: ignored

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    # get unique indices
    tmp1 = tf.reshape(skip_ids, shape=(-1,))
    uniques, idx, counts = tf.unique_with_counts(tmp1)
    uniques_ids = tf.expand_dims(uniques, axis=1)

    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(uniques_ids),
        indices = uniques_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    
    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Script:' + '\n\n'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

# Import datetime class from datetime module
from datetime import datetime

now = str(datetime.now())

filename = now + '.txt'

a = open(filename, 'w')

list_of_tuples = result[0].numpy().decode('utf-8'), '\n\n' + '_'*80
temp = " ".join(map(str, list_of_tuples))
a.write(temp)
a.close


# print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

Ceza:
                                                                        70.


A chain are ractor closes in close. Thirmback is porneging, but steaming
water as someone sleeps aside; it developing inside the camera.

Catherine's screen listens to a halt-eyed Pable.

    JULIA (CONT'D)
   (dryating at the door)
  How do you didn't know? I don't go for
  him!

The churney's open light on an animal has landing,
revealing the (with rat) And the strobes scatter
          about...

Matt and An ABSENT-LOVE POLICE on the bar, feeling the
Gate For water...

Their last fade is hearing the word with concroting fingers.

                    

                                                                    CUT.

                                   MOTHER
                    Sorry to act out the guy I'm
                    not leaving, Matt.
          His cry--Ared down there this as.. I don't know.

                     NEYTIRI
                  I asked you hear real fraiguit
              

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 146442 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.24-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
!nvidia-smi


Wed Feb 17 20:21:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------